# GLM4大模型微调入门

作者：林泽毅

教程文章：https://zhuanlan.zhihu.com/p/702608991

显存要求：40GB左右  

实验过程看：https://swanlab.cn/@ZeyiLin/GLM4-fintune/runs/eabll3xug8orsxzjy4yu4/chart

## 1.安装环境

本案例测试于modelscope==1.14.0、transformers==4.41.2、datasets==2.18.0、peft==0.11.1、accelerate==0.30.1、swanlab==0.3.10、tiktoken==0.7.0

In [1]:
%pip install torch swanlab modelscope transformers datasets peft pandas accelerate tiktoken --quiet


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


如果是第一次使用SwanLab，则前往[SwanLab](https://swanlab.cn)注册账号后，在[用户设置](https://swanlab.cn/settings/overview)复制API Key，如果执行下面的代码：

In [2]:
!swanlab login

swanlab: You are already logged in. Use `swanlab login --relogin` to force relogin.


## 2. 数据集加载

1. 在[zh_cls_fudan-news - modelscope](https://modelscope.cn/datasets/huangjintao/zh_cls_fudan-news/files)下载train.jsonl和test.jsonl到同级目录下。

<img src="../assets/dataset.png" width=600>

2. 将train.jsonl和test.jsonl进行处理，转换成new_train.jsonl和new_test.jsonl

In [3]:
# 2.将train.jsonl和test.jsonl进行处理，转换成new_train.jsonl和new_test.jsonl

import json
import pandas as pd
import os

def dataset_jsonl_transfer(origin_path, new_path):
    """
    将原始数据集转换为大模型微调所需数据格式的新数据集
    """
    messages = []

    # 读取旧的JSONL文件
    with open(origin_path, "r") as file:
        for line in file:
            # 解析每一行的json数据
            data = json.loads(line)
            context = data["text"]
            catagory = data["category"]
            label = data["output"]
            message = {
                "instruction": "你是一个文本分类领域的专家，你会接收到一段文本和几个潜在的分类选项，请输出文本内容的正确类型",
                "input": f"文本:{context},类型选型:{catagory}",
                "output": label,
            }
            messages.append(message)

    # 保存重构后的JSONL文件
    with open(new_path, "w", encoding="utf-8") as file:
        for message in messages:
            file.write(json.dumps(message, ensure_ascii=False) + "\n")


# 加载、处理数据集和测试集
train_dataset_path = "train.jsonl"
test_dataset_path = "test.jsonl"

train_jsonl_new_path = "new_train.jsonl"
test_jsonl_new_path = "new_test.jsonl"

if not os.path.exists(train_jsonl_new_path):
    dataset_jsonl_transfer(train_dataset_path, train_jsonl_new_path)
if not os.path.exists(test_jsonl_new_path):
    dataset_jsonl_transfer(test_dataset_path, test_jsonl_new_path)

train_df = pd.read_json(train_jsonl_new_path, lines=True)[:1000]  # 取前1000条做训练（可选）
test_df = pd.read_json(test_jsonl_new_path, lines=True)[:10]  # 取前10条做主观评测

## 3. 下载/加载模型和tokenizer

In [4]:
from modelscope import snapshot_download, AutoTokenizer
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch

# 在modelscope上下载Qwen模型到本地目录下
model_dir = snapshot_download("ZhipuAI/glm-4-9b-chat", cache_dir="./", revision="master")

# Transformers加载模型权重
tokenizer = AutoTokenizer.from_pretrained("./ZhipuAI/glm-4-9b-chat/", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./ZhipuAI/glm-4-9b-chat/", 
                                             device_map="auto",
                                             torch_dtype=torch.bfloat16,
                                             trust_remote_code=True)
model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法

2024-10-30 06:50:33,541 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

## 4. 预处理训练数据

In [5]:
def process_func(example):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = 384
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
        f"<|im_start|>system\n你是一个文本分类领域的专家，你会接收到一段文本和几个潜在的分类选项，请输出文本内容的正确类型<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False,
    )
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = (
        instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    )
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = (
        [-100] * len(instruction["input_ids"])
        + response["input_ids"]
        + [tokenizer.pad_token_id]
    )
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [6]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df)
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## 5. 设置LORA

In [7]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "activation_func", "dense_4h_to_h"],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,  # Dropout 比例
)

model = get_peft_model(model, config)

## 6. 训练

In [8]:
args = TrainingArguments(
    output_dir="./output/GLM4-9b",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=2,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

In [9]:
from swanlab.integration.huggingface import SwanLabCallback
import swanlab

swanlab_callback = SwanLabCallback(
    project="GLM4-fintune",
    experiment_name="GLM4-9B-Chat",
    description="使用智谱GLM4-9B-Chat模型在zh_cls_fudan-news数据集上微调。",
    config={
        "model": "ZhipuAI/glm-4-9b-chat",
        "dataset": "huangjintao/zh_cls_fudan-news",
    },
)

/tmp/ipykernel_370/1911955071.py:1: DeprecationWarning: The module 'huggingface' is deprecated and will be removed in future versions. Please update your imports to use 'transformers' instead.
  from swanlab.integration.huggingface import SwanLabCallback


In [10]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)

trainer.train()


# ====== 训练结束后的预测 ===== #

def predict(messages, model, tokenizer):
    # device = "cuda"
    device = "cuda"
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)

    return response
    

test_text_list = []
for index, row in test_df.iterrows():
    instruction = row["instruction"]
    input_value = row["input"]

    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"},
    ]

    response = predict(messages, model, tokenizer)
    messages.append({"role": "assistant", "content": f"{response}"})
    result_text = f"{messages[0]}\n\n{messages[1]}\n\n{messages[2]}"
    test_text_list.append(swanlab.Text(result_text, caption=response))

swanlab.log({"Prediction": test_text_list})
swanlab.finish()

swanlab: Tracking run with swanlab version 0.3.23                                  
swanlab: Run data will be saved locally in /opt/app-root/src/LLM-Finetune/notebook/swanlog/run-20241030_065311-a3b1799d
swanlab: 👋 Hi alanliuxiang, welcome to swanlab!
swanlab: Syncing run GLM4-9B-Chat to the cloud
swanlab: 🌟 Run `swanlab watch /opt/app-root/src/LLM-Finetune/notebook/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@alanliuxiang/GLM4-fintune
swanlab: 🚀 View run at https://swanlab.cn/@alanliuxiang/GLM4-fintune/runs/cwa0yg7u3qlzu0dvh2m12



[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,343.863300
20,3.238900
30,1.751800
40,1.039000
50,11.853500
60,0.051000
70,3.474300
80,0.261500
90,0.000400
100,0.004000


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Computerisuianatahone for all forereio- isianw: forma, oninaipapfinaianw:4 andoni: for aaseatuotainonaere.::RisuXuuduXfintandi:alatere.::Gon andonusi: a遇了,ianatuntinaereosereonu-: isatu2:one,ereereodetumat lou0, inianw: onus. in.::: andi:2, foram-2.. foramuiraone, inanacone,usaioneereereetatunton a ... inaspy::Séis foru32:uahat foruahunone.4 andusianw:icere,elonu: forolved toainuinedumonuöainimaianatontelis isereonuudere.4 inisis andonu. forrestetereahniereere.::Sere.,atere.,maamacet.,awcunif. deereetosatuaininais foruöonuereio- foruainatlocumoni: forisanu3 inionacuone forere. for theatuł: forigraphy isip:::awallonipiron foru tilunhapeti:iaitaet,ereereere.,awm4ere. foric/. forentsere. atton  OB:. inund:::Cul. andesatuawu deianimu,atmani:is for theatisatfumanif forangu deere.::Gund:::Risatam Iawahone inu2 deereility..4 deereereereereeti:am4 deere.,attranhirereereinaoni:is for a�ularu3.::A. foruainio.4 for theonu/ deereenaereudinaereere.  wasipierce isieatiair:.::Cainu,4 for a obsc: form